In [ ]:
import scipy
import scipy.integrate as si
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import os,sys 
os.chdir("..")

In [ ]:
from torchquad.utils.benchmark_utils import _runtime_measure as runtime_measure
from torchquad.utils.benchmark_utils import _get_integral as get_integral
from torchquad.plots import plot_runtime as plt_rt
from torchquad.plots import plot_convergence as plt_conv
from torchquad.integration.monte_carlo import MonteCarlo 
from torchquad.integration.trapezoid_1D import Trapezoid1D
from torchquad.utils.enable_cuda import enable_cuda
from torchquad.tests import trapezoid_1D_test
import torch

## Enable GPU usage

In [ ]:
enable_cuda()

## Set precision

In [ ]:
np.set_printoptions(precision=10)
torch.set_printoptions(precision=10)

## Some test functions

In [ ]:
def f1(x):
    return 4 - torch.pow(x,2)

def f1_2(x):
    return  torch.pow(x,5) - 4* torch.pow(x,4) + 23* torch.pow(x,3) + 4* torch.pow(x,2) - 3*x + 1

def f_x(x):
    return x

## Selecting test function and integration method

In [ ]:
f_test_dict={'f1' : f1, 'f1_2' : f1_2, 'f_x' : f_x}

In [ ]:
f_test_used = 'f1'
f_test = f_test_dict[f_test_used]

In [ ]:
scipy_method = scipy.trapz #scipy_based integration method
integral_torch_method = Trapezoid1D() #Torchquad integration method

## Selecting number of integration points and average measurements per step

In [ ]:
N_points = [10, 100, 1000, 10000] #Number of iteration tests
iterations = 10 #Number of iterations used to take an average runtime measurement

In [ ]:
ground_through_dict = {'f1' : 22/3 * np.ones(len(N_points)), 'f1_2' : 46/15 * np.ones(len(N_points)), 'f_x': np.zeros(len(N_points))}

## Testing runtime performance

In [ ]:
scipy_method_runtime = runtime_measure(method=scipy_method, scipy_based=True, fn=f_test, N=N_points, iterations=iterations)

In [ ]:
torch_method_runtime = runtime_measure(method=integral_torch_method.integrate, scipy_based=False, fn=f_test, N=N_points, iterations=iterations)

In [ ]:
plt_rt.plot_runtime([N_points, N_points], [scipy_method_runtime, torch_method_runtime], labels=['scipy_method', 'torch_method'])

## Measuring convergence rate

In [ ]:
scipy_method_integral = get_integral(method=scipy_method, scipy_based=True, fn=f_test, N=N_points)

In [ ]:
torch_method_integral = get_integral(method=integral_torch_method.integrate, scipy_based=False, fn=f_test, N=N_points)

In [ ]:
plt_conv.plot_convergence([N_points, N_points], [scipy_method_integral, torch_method_integral], ground_through_dict[f_test_used] , ['scipy_method', 'torch_method'], dpi=150)